In [24]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Macbook_FZN/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
data_train=pd.read_csv('/Users/Macbook_FZN/Documents/AMAN/Practicum/Dataset/Train.tsv',delimiter='\t',encoding='utf-8')

In [35]:
data_train=data_train.drop(data_train.columns[[0,5,6]],axis = 1)

In [36]:
data_train.head(10)

,drugName,condition,review,rating
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0
5,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0
7,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0
8,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0
9,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0


In [37]:
data_train['drugName']=data_train['drugName'].str.lower() 

In [38]:
data_train.head(10)

,drugName,condition,review,rating
0,valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0
1,guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0
2,lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0
3,ortho evra,Birth Control,"""This is my first time using any form of birth...",8.0
4,buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0
5,cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0
6,levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0
7,aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0
8,keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0
9,ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0


In [42]:
data_train['drugName'].nunique()

3436

In [43]:
#Anonymization of drugName

data_train['drugName']='medicine' + pd.Series(pd.factorize(data_train['drugName'])[0] + 1).astype(str)

In [47]:
data_train.head(10)

,drugName,condition,review,rating
0,medicine1,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0
1,medicine2,ADHD,"""My son is halfway through his fourth week of ...",8.0
2,medicine3,Birth Control,"""I used to take another oral contraceptive, wh...",5.0
3,medicine4,Birth Control,"""This is my first time using any form of birth...",8.0
4,medicine5,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0
5,medicine6,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0
6,medicine7,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0
7,medicine8,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0
8,medicine9,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0
9,medicine10,Birth Control,"""I had been on the pill for many years. When m...",8.0


In [53]:
data_train.tail(5)

,drugName,condition,review,rating
161292,medicine788,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0
161293,medicine269,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0
161294,medicine832,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0
161295,medicine343,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10.0
161296,medicine171,"Constipation, Chronic","""I&#039;ve had chronic constipation all my adu...",9.0


In [54]:
data_train['review'].isnull().sum().sum()

0

In [55]:
def rem_sw(df):
    # Downloading stop words
    stop_words = set(stopwords.words('english'))

    # Removing Stop words from training data
    count = 0
    for sentence in df:
        sentence = [word for word in sentence.lower().split() if word not in stop_words]
        sentence = ' '.join(sentence)
        df.loc[count] = sentence
        count+=1
    return(df)

def rem_punc(df):
    count = 0
    for s in df:
        cleanr = re.compile('<.*?>')
        s = re.sub(r'\d+', '', s)
        s = re.sub(cleanr, '', s)
        s = re.sub("'", '', s)
        s = re.sub(r'\W+', ' ', s)
        s = s.replace('_', '')
        df.loc[count] = s
        count+=1
    return(df)

def lemma(df):

    lmtzr = WordNetLemmatizer()

    count = 0
    stemmed = []
    for sentence in df:    
        word_tokens = word_tokenize(sentence)
        for word in word_tokens:
            stemmed.append(lmtzr.lemmatize(word))
        sentence = ' '.join(stemmed)
        df.iloc[count] = sentence
        count+=1
        stemmed = []
    return(df)

In [56]:
##################################### Removing Stop words and convert to lower case from training data ##################################
nltk.download('stopwords')
data_train['review'] = rem_sw(data_train['review'])



/Users/Macbook_FZN/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [60]:
data_train['review'].head(5)

0     it side effect take combination bystolic mg f...
1     my son halfway fourth week intuniv became con...
2     i used take another oral contraceptive pill c...
3     this first time using form birth control i m ...
4     suboxone completely turned life around feel h...
Name: review, dtype: object

In [59]:
###################################### Removing punctuations from Train set ##################################
import re
data_train['review'] = rem_punc(data_train['review'])

/Users/Macbook_FZN/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [62]:
data_train['review'].head(10)

0     it side effect take combination bystolic mg f...
1     my son halfway fourth week intuniv became con...
2     i used take another oral contraceptive pill c...
3     this first time using form birth control i m ...
4     suboxone completely turned life around feel h...
5     nd day mg started work rock hard erections ho...
6     he pulled out cummed bit me took plan b hours...
7     abilify changed life hope zoloft clonidine fi...
8     nothing problems keppera constant shaking arm...
9     i pill many years doctor changed rx chateal e...
Name: review, dtype: object

In [63]:
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
data_train['review'] = lemma(data_train['review'])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Macbook_FZN/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/Users/Macbook_FZN/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [1]:
data_train['review'].head(10)

NameError: name 'data_train' is not defined